In [54]:
from nutils import mesh, function, plot, _
import numpy as np
import scipy as sp
from numpy import linalg as LA

# Poisson solver using NUTILS
## Domain
First the geometry is made. For instance we get the domain and the geometry of the physical space $\Omega$ and its basis together with the geometry of the parameter space $\tilde{\Omega}$ calling the function $setup\_domain$. As input we give the desired number of elements in $\xi$- and $\eta$-direction $n$ and $m$ along with the order of the basis functions $p$ and $q$ in $\xi$- and $\eta$-direction respectively.

Also we must give the function the mapping from the parameterspace $\tilde{\Omega}=[0,1]\times[0,1]$ to the physical space $\Omega$, given as a function $physicalspace$ taking $\xi$ and $\eta$ as input, returning $x$ and $y$.

## Basis functions
Assuming there is no repeated knots except at the boundary (because of open knotvectors) there will be $n+1$ and $m+1$ unique knots in the $\xi$- and $\eta$-direction respectively resulting in a total of $n+1+2p$ and $m+1+2q$ knots in the two directions respectively, where $p$ is the order of the basisfunctions in the $\xi$-direction and $q$ is the order of the basisfunctions in the $\eta$-direction. Hence as a general knotvector is of length $\tilde{n}+\tilde{p}+1$ where $\tilde n$ is the number of basis functions and $\tilde p$ is the polynomial degree, we have $n+p$ number of basis functions $N_{i,p}(\xi)$ in the $\xi$-direction and $m+q$ number of basis functions $N_{j,q}(\eta)$ in the $\eta$-direction, resulting in a total amount of $(n+p)(m+q)$ basis functions $\tilde N_A(\xi,\eta)=N_{i,p}(\xi)N_{j,q}(\eta)$ of polynomial degree $p+q$



In [55]:
def annulus_function(xi,eta):
    # Gives x,y in physical pace represented by xi and eta
	return (eta+1)*(function.cos(xi*np.pi/2), function.sin(xi*np.pi/2))

def setup_domain(n,m,p,q,physicalspace):
	# n+1 grid points in x direction
	# m+1 grid points in y direction
	# p,q corrensponding polynomial degree of basis functions
	# physicalspace is a function taking xi and eta as input, returning geom
	# problem_name - name of the problem
	# plotting - boolean for plotting solution, exact and approximated, and pointwise error or not

	### Geometry - obtain basis for parameter space and a representation of the physical space
	#1 - Make parameter space
	xs=np.linspace(0,1,n+1)
	ys=np.linspace(0,1,m+1)
	domain, grid_geom=mesh.rectilinear([xs,ys])
	#2 - Make basis
	#now xs and ys will be our knot vecs in each spatial direction without any repeated knots. Hence the open knotvec
	# will be of size n+1+2p(m+1+2q) and hence the number of basis functions in each direction is N=n+1+2p-1-p=n+p, M=(m+q)
	basis = domain.basis('spline',degree=(p,q)) # What is important for this basis, how to choose degree
	# Physical space - geom
	xi,eta=grid_geom
	geom=physicalspace(xi,eta)
	x,y=geom
	#Set f and exact solution u
	return domain, grid_geom,geom,basis

## Poisson Problem
We solve Poisson's equation with Dirichlet boundary conditions.
\begin{equation}
    \begin{split}
        -\Delta u &= f \quad \mbox{on } \Omega \\
        u &= g \quad \mbox{on } \Omega \\
    \end{split}
\end{equation}
The poisson solver takes the polynomial degree $p$ and $q$ of the basis functions and the domain and the geometry of the physical space $\Omega$ as input. Also it needs a function $f$ for the right hand side and a function $bc$ for the boundary conditons, here assuming the whole boundary can be described with a single function.
### Solving the problem
Solving the problem is straight forward from here, assemble the $K$ matrix from my notes on $\textit{A simple Poisson solver}$, here named matrix. Project the boundary conditions onto the basis to identify the basis functions connected to the boundary points (control points on the boundary). $I$ will contain the indices for every basis function connected to the boundary in an array. Compute the right hand side and then compute the linear system using the Conjugate Gradient method resulting in $lhs$ better known as $u$. Here a lifting function is used by the nutils-function $solve$, see section $\textit{Boundary conditions}$ for further details.
$poisson()$ returns the solution of the system, the stiffness matrix and the array $I$ keeping track of the control points/basis functions connected to the boundary.

### Boundary conditions
To deal with nonhomogeneous boundary conditions a lifting function is introduced, that means we solve the weak problem by inserting $u=u_0+u_g$ into the weak formulation given in $\textit{A simple Poisson solver}$. Here $u_0$ is 0 at the boundary, i.e. $u_0\in V$, meaning we find it by the same means as described in $\textit{A simple Poisson solver}$. The lifting function $u_g$ should then obtain the value of $u$ at the boundary, what happens on the interior doesn't matter as we will see, $u_g\in U$.
\begin{equation}
    \begin{split}
        a(N_j,N_i)u_{0i}&=F(N_j)-a(N_j,N_i)u_{gi}\\
        \mathbf{A}\mathbf{u_0} &=\mathbf{F}-\mathbf{A}\mathbf{u_g} \\
    \end{split}
\end{equation}
where $a(\cdot,\cdot)$ and $F(\cdot)$ are as given in $\textit{A simple Poisson solver}$ and $u_0$ is the solution of the homogeneous Dirichlet problem and $u_g$ is the lifting function taking the value of the boundary conditions at the boundary. 

In $poisson()$ we solve the linear system using the nutils-function $.solve$. For nonhomogeneous boundary conditions this function uses a lifting function, and what it basically does behind the scenes is


\begin{equation}
    \begin{split}
        \tiny{1}\quad&\mbox{Set all entries of }cons\mbox{ corresponding to interior control points to 0}\\ 
        \tiny{2}\quad&\mbox{Reduce matrix, rhs and cons removing rows and columns connected to boundary points}\\
       \tiny{3}\quad &\mbox{solve }u_0=matrix^{-1}(rhs-matrix\times cons)\mbox{ using Conjugate gradient method}\\
       \tiny{4}\quad &\mbox{Increase } u_0\mbox{ back to original dimensions with 0 entries for all boundary points}\\
       \tiny{5}\quad &\mbox{solve } u=u_0+cons,
    \end{split}
\end{equation}


where $u$ is our $lhs.$ We need to reduce the system to sort of "free" dofs as our matrix is singular. In the third line we see that for all interior points we subtract the value of $u_g$ or $cons$ from $u_0$ and in the fifth line we add them back to $u_0$ together with its value at the boundary points, and hence it doesn't matter what value the lifting function takes on the interior. 
### Gaussian quadrature 
As mentioned above the polynomial degree of the basis functions is $p$ and $q$ in $x$- and $y$-direction respectively and this gives us basisfunctions of degree $p+q$. For the integral 
\begin{equation}
\iint_{\Omega}\nabla N_A(\xi,\eta)\cdot\nabla N_B(\xi,\eta)|J(\xi,\eta)|\mbox{ d}\xi\mbox{d}\eta 
\end{equation}
where $N_i$ is our basis functions of degree $p+q$, $p$ in $\xi$-direction and $q$ in $\eta$-direction, i.e. $N_i\in P^{p,q}$. We separate the directions because we need the right amount of gaussian points to integrate exactly in both $\xi$ and $\eta$. The gradient is a vector, $\nabla N_i\in [P^{(p-1),q},P^{p,q-1}]$ and hence $\nabla N_A(\xi,\eta)\cdot\nabla N_B(\xi,\eta)\in P^{2p,2q}$. Recaling the notion of the Jacobian from $\textit{A simple poisson solver}$ and the notion of $x$ and $y$ we get that
\begin{equation}
  |J|=\frac{\partial x}{\partial \xi}  \frac{\partial y}{\partial \eta} - \frac{\partial x}{\partial \eta}\frac{\partial y}{\partial \xi} \in P^{2p-1,2q-1} .
\end{equation}
This results in our integrand $\nabla N_A(\xi,\eta)\cdot\nabla N_B(\xi,\eta)\left|J(\xi,\eta)\right|\in P^{4p-1,4q-1}$. This means that to integrate exactly we need $2p$ gauss points in $\xi$-direction and $2q$ points in $\eta$-direction.

The second integral is of course dependent on our function $f$
\begin{equation}
    \iint_{\Omega}f(x(\xi,\eta),y(\xi,\eta)) N_B(\xi,\eta)|J|\mbox{ d}\xi\mbox{d}\eta
\end{equation}
but for $f$ with lower or equal degree as $P^{p,q}$ (since the degree of $N_B(\xi,\eta)|J|$ is $P^{3p-1,3q-1}$), we can just use the same number of gaussian points for simplicity. For our testcase on the other hand, as we will come to, this will not be enough to integrate exactly as the sine function is not exactly represented by polynomials. Still, for now we are using the same amount of gaussian points.

In [56]:
def poisson(p,q,domain,geom,basis,f,bc):
	# p,q corrensponding polynomial degree of basis functions
	# geom gives the geometry for which the problem is solved
	# basis is the basis functions for the space
	# f is the rhs function
	# bc gives the boundary conditions
	# Returns sol,matrix,I where sol is the approximate solution (lin. comb of basisfunctions using the controlpoints)
	# matrix is the stiffness matrix, and I gives the indicies in the stiffness matrix corresponding to internal points
	### Assemble
	#Assemble stiffness matric
	grad = basis.grad(geom)
	outer = function.outer(grad,grad)
	integrand = outer.sum(-1)
	degree=np.max([p,q])
	gauss='gauss'+str(int(2*(degree))) #Not sure, multiply with 2 for now, being the order of the gradient of my basis function in one direction
	matrix = domain.integrate(integrand, geometry=geom,ischeme=gauss)
	# b.c.
	x,y=geom
	cons=domain.boundary.project(bc(x,y),onto=basis,geometry=geom,ischeme=gauss)
	#Indicies of all elements not belonging to boundary points
	I, = np.where(np.isnan(cons))
	# Compute rhs 
	rhs = domain.integrate(f(x,y)*basis,geometry=geom,ischeme=gauss)
	# compute lhs
	lhs = matrix.solve(rhs, constrain=cons, solver='cg')
	sol=basis.dot(lhs)
	return sol,matrix,I

## Test problem
As a test problem I choose to work with $u(x,y)=\sin(2x+2y)$. For this problem the physical domain chosen is a quarter annulus with inner radius $r_1=1$ and outer radius $r_2=2$. 

### Domain setup
For the parameter space $[0,1]\times[0,1]=\tilde\Omega$ the mapping from $\tilde{\Omega}$ to $\Omega$ is given by
\begin{equation}
 \left[ {\begin{array}{c}
    x \\
    y \\
 \end{array} }\right ] =
 \left[ {\begin{array}{c}
    (\xi+1)\cos\left(\frac{\pi}{2}\eta\right) \\
    (\xi+1)\sin\left(\frac{\pi}{2}\eta\right) \\
 \end{array}}\right ],
\end{equation}
where $(\xi,\eta)\in\tilde\Omega$. See a plot of the two spaces bellow. We choose the following parameters.


In [57]:
#### Setup domain 
n=15
m=15
p=4
q=4

domain, grid_geom, geom, basis=setup_domain(n,m,p,q,annulus_function)

parameter_points=domain.elem_eval(grid_geom,ischeme='bezier9', separate=True)
physical_points=domain.elem_eval(geom,ischeme='bezier9', separate=True)

with plot.PyPlot('mesh_parameterspace',index=0) as plt:
    plt.mesh(parameter_points)
    plt.title(r"parameter space with $m\times n$="+str(m)+r"$\times$"+str(n)+" elements")

    
with plot.PyPlot('mesh_physicalspace',index=0) as plt:
    plt.mesh(physical_points)
    plt.title(r"physical space with $m\times n$="+str(m)+r"$\times$"+str(n)+" elements")
    
#### Setup test problem
# Function f in the poisson problem
def f1(x,y):
	return 8*(function.sin(2*x+2*y))

# Boundary conditions and in fact the exact solution of the problem
def bc1(x,y):
	return function.sin(2*x+2*y)


elem_eval > created ndarray(18225,2)
elem_eval > created ndarray(18225,2)
mesh_parameterspace000.png
mesh_physicalspace000.png



![Parameterspace](mesh_parameterspace000.png)

![Physicalspace](mesh_physicalspace000.png)

## Solution and error analysis

$poisson()$ computes the control points $\mathbf{u_h}$ of the approximated solution of $u$ and hence we can compute the solution $u_h(\xi, \eta)=\sum_{i=1}^{nm}N_i(\xi,\eta)u_{hi}=\mathbf{N}^\top\mathbf{u_h}$. By the mapping $x=\sum_{i=1}^{nm}N_i(\xi,\eta)x_i$, $y=\sum_{i=1}^{nm}N_i(\xi,\eta)y_i$ it is possible to evaluate both the value of $u_h$ and our exact test function $u=\sin(2x+2y)$ over the space $\Omega$ and hence we look at the pointwise error $|u-u_h|$ and plot this. Also it is interesting to look at the norm of the error, choosing the $L^2$-norm, $\lVert u-u_h \rVert_2^2. As the $L^2$-norm is an integral we are using gaussian quadrature as earlier. Even if it is not possible to integrate the sine function exactly we still want to increase the number of quadrature points to increase accuracy.

The following function computes the pointwise error and the error in the $L^2$-norm, and returns this error. For $plotting=1$ the function plots the exact solution of the problem, the approximate solution and the pointwise error.



In [58]:
def error_analysis(sol, u,domain, geom,problem_name='test',plotting=0):
	#sol is the controlpoints evaluated at its basis
	# u is the exact solution, geom is the geometry
	# problem_name is the filename/name of the plots of the results, test by default
	# Plotting is a boolean for plotting, 1, the results, or not, 0
	x,y=geom
	# u is the exact solution
	exact_points,exact_color = domain.elem_eval([geom,u(x,y)],ischeme='bezier9', separate=True)
	### error analysis
	# L2 norm of u-uh
	integrand=(sol-u(x,y))**2
	gauss_error='gauss21' # choose high order gauss for this integral, to get better "precission" (giving 11 gauss points in each direction)
	L2_error=domain.integrate(integrand, geometry=geom,ischeme=gauss_error)
	
	#pointwise error |u-uh|
	pointwise_error=np.absolute(sol-u(x,y))
	error_points,error_color = domain.elem_eval(
	[geom,pointwise_error],
	ischeme='bezier9',separate=True
	)

	# approximate solution u_h
	sol_points, sol_color = domain.elem_eval(
	[geom,sol],
	ischeme='bezier9',separate=True
	)
	#Plotting
	if plotting:
		with plot.PyPlot(problem_name+'_u_approx',index=0) as plt: 
			plt.mesh(sol_points,sol_color)
			plt.title('$u_h$')
			plt.colorbar()
		with plot.PyPlot(problem_name+'_u_exact',index=0) as plt: 
			plt.mesh(exact_points,exact_color)
			plt.title('$u$')
			plt.colorbar()
		with plot.PyPlot(problem_name+'_pointwise_error',index=0) as plt:
			plt.mesh(error_points,error_color)
			plt.title('$|u-u_h|$')
			plt.colorbar()
	return L2_error


### Plotting
name = "poisson_FEM" # Name of problem
## Solving the problem
sol,K,I=poisson(p,q,domain,geom,basis,f1,bc1)
## Computing error
L2_error=error_analysis(sol, bc1,domain, geom, name, 1)
print("The L^2-error squared is ", L2_error)

project > solving system > solving system using sparse direct solver
project > constrained 72/361 dofs, error 6.02e-07/area
solving system > solving system using sparse direct solver
elem_eval > created ndarray(18225,2), ndarray(18225)
elem_eval > created ndarray(18225,2), ndarray(18225)
elem_eval > created ndarray(18225,2), ndarray(18225)
poisson_FEM_u_approx000.png
poisson_FEM_u_exact000.png
poisson_FEM_pointwise_error000.png
The L^2-error squared is  3.31834504139e-12


![u_approx](poisson_FEM_u_approx000.png)
![u_exact](poisson_FEM_u_exact000.png)
![pointwise_error](poisson_FEM_pointwise_error000.png)


## Convergence

It is a good way to verify that your method is stable producing convergence plots. $convergence()$ is a function calling $error\_analysis()$ for different values of $n,m,p$ and $q$ and storing all the different values of the error in a $.npz$ file. The function increases the number of elements $nm$, increasing $n$ and $m$ at the same time with the same size, while keeping the polynomial degree $p=q$ constant. The we increase the polynomial degree and repeat everything. $n$ and $m$ are increased as powers of $2$ with $nmin$ the starting power and $nmax$ the last power.

In [41]:
def convergence(filename,physicalspace,f,bc,pmin=1,pmax=4,nmin=0,nmax=6):
	# plot the convergence of the error for increasing number of elements, {2^nmin-2^nmax}, for different polynomial degree pmin-pmax,
	# saves the results in a npz file, can be visualized using plot_convergence()
	# increasing n, m with the same amount at the same time for each step, same for p and q
	# pmin,pmax is the min/max degree of the basis functions
	# n+1 gives number of grid points in one direction, hence nmin, nmax is the min/max number of gridpartitions
	# notice n is a power of two, nmin, nmax gives the min and max powers
	# Have assumed bc to be the exact solution over the whole domain for now, use this on boundary and also as exact solution
	vec=np.arange(nmin,nmax+1)
	n=2**vec
	m=n
	p=np.arange(pmin,pmax+1)
	q=np.arange(pmin,pmax+1)
	error=np.zeros((len(p),len(n)))
	label=np.empty((len(p),),dtype=object)
	for i in range(0,len(p)):
		label[i]="p=q="+str(p[i])
		for j in range(0,len(n)):
			#Domain and basis setup
			domain, grid_geom,geom,basis=setup_domain(n[j],m[j],p[i],q[i],physicalspace)
			# Solve problem (assemble matrix, solve system)
			sol,_,_=poisson(p,q,domain,geom,basis,f,bc)
			# compute L2 error
			error[i,j]=error_analysis(sol, bc,domain, geom)
	# Writing results to file
	np.savez(filename,n=n,error=error,label=label)


    
#### Set the parameters
pmin=1
pmax=5
nmin=0
nmax=6

convergence(name,annulus_function,f1,bc1,pmin,pmax,nmin,nmax)

project > solving system > solving system using sparse direct solver
project > constrained 4/4 dofs, error 7.22e-02/area
solving system > solving system using sparse direct solver
solving system > RuntimeWarning: invalid value encountered in double_scalars
  In C:\Users\Lars Snekkerhaugen\Anaconda3\lib\site-packages\nutils\matrix.py:41
elem_eval > created ndarray(81,2), ndarray(81)
elem_eval > created ndarray(81,2), ndarray(81)
elem_eval > created ndarray(81,2), ndarray(81)
project > solving system > solving system using sparse direct solver
project > constrained 8/9 dofs, error 2.68e-02/area
solving system > solving system using sparse direct solver
elem_eval > created ndarray(324,2), ndarray(324)
elem_eval > created ndarray(324,2), ndarray(324)
elem_eval > created ndarray(324,2), ndarray(324)
project > solving system > solving system using sparse direct solver
project > constrained 16/25 dofs, error 1.64e-02/area
solving system > solving system using sparse direct solver
elem_eval > 

integrate > elem 1819/4096 (44%)
solving system > solving system using sparse direct solver
elem_eval > elem 2359/4096 (57%)
elem_eval > created ndarray(331776,2), ndarray(331776)
integrate > elem 621/4096 (15%)
integrate > elem 1894/4096 (46%)
elem_eval > elem 2004/4096 (48%)
elem_eval > created ndarray(331776,2), ndarray(331776)
elem_eval > elem 2422/4096 (59%)
elem_eval > created ndarray(331776,2), ndarray(331776)
project > solving system > solving system using sparse direct solver
project > constrained 16/25 dofs, error 3.83e-04/area
solving system > solving system using sparse direct solver
elem_eval > created ndarray(81,2), ndarray(81)
elem_eval > created ndarray(81,2), ndarray(81)
elem_eval > created ndarray(81,2), ndarray(81)
project > solving system > solving system using sparse direct solver
project > constrained 20/36 dofs, error 4.79e-04/area
solving system > solving system using sparse direct solver
elem_eval > created ndarray(324,2), ndarray(324)
elem_eval > created ndarr

### Convergence plot
To visualize the results we call $plot\_convergence()$ with the correct filename.

In [59]:
def plot_convergence(filename):
	# Loads a .npz file with element 'n' a 1xm vector, where each entry is the number of elements used in one direction
	# element 'error' of size nxm where the error is plotted for n different polynomial degrees and m is still different number of elements
	# element 'label' 1xn vector, with n labels, one for each polynomial degree 
	with np.load(filename+'.npz') as data:
		data=np.load(filename+'.npz')
		n=data['n']
		error=data['error']
		label=data['label']
	row,col=error.shape
	fig,ax=plt.subplots()
	for i in range(0, row):
	    ax.plot(n, error[i,:],marker='o', label=label[i])
	#plt.plot(n.transpose(),error.transpose(),label=label[0,:]) #https://stackoverflow.com/questions/11481644/how-do-i-assign-multiple-labels-at-once-in-matplotlib
	ax.set_title('Convergence plot - logscale')
	ax.set_yscale('log')
	ax.set_xscale('log')
	ax.set_ylabel('$||u-u_h||_{L^2}^2$')
	ax.set_xlabel('n=m')
	ax.legend(loc='upper right')
	fig.savefig(filename+'.png')
	plt.show()
    
    
### Plot convergence results
plot_convergence(name)

![convergence](poisson_FEM.png)

# Reduced basis
## Introduction to reduced basis methods
Hvorfor
## Theory on reduced basis method

Let's denote our original basis $\mathbf{N}=\{N_i\}$ for $i=1,\dots,m$. From our stiffness matrix we choose $n<m$ eigenvalues $\lambda_j$ for $j=1,\dots,n$ and the corresponding eigenvectors $\mathbf{v_j}$ of dimension $m\times 1$. Typically we want to choose the largest eigenvalues as these corresponds to the energy in our system. This yields a matrix $\mathbf{V}$ of size $m\times n$, where column $j$ corresponds to eigenvector $\mathbf{v_j}$. Now we construct new basis functions $\tilde{N}_j$ as a linear combinations of our old basis functions $N_i$ with coefficients $v_{ij}$. Hence we get 
\begin{equation}
\tilde{\mathbf{N}}=\mathbf{V^\top}\mathbf{N}
\end{equation}
where $\tilde{\mathbf{N}}=\{\tilde{N}_j\}$ is a vector of basis functions, $\mathbf{V}=\{v_{ij}\}$ is a matrix with mapping coefficients and $i=1,\dots,m$, $j=1,\dots,n$.

Sette opp weak formulation med reduced basis. Bruk matrisen A


In [60]:
# Could be made much easier using .eigh as this gives eigenvalues and vectors in descending order

def reduce_basis(K, percent,J, basis):
	# Select eigenvectors from K corresponding to x% of the total sum of the eigenvalues, choosing only the largest eigenvalues
	# and return the corresponding eigenvectors in a matrix with a eigenvector for each column
	# J gives the indicies in the stiffness matrix corresponding to internal points
	K=K.toarray()
	# save the size of the original stiffness matrix
	row,col=K.shape
	# Remove rows and columns belonging to boundary points
	K=K[np.ix_(J,J)]
	eigenval,w=LA.eig(K)
	# Total sum of all eigenvalues
	total = eigenval.sum()
	# Keep track of the sum of all eigenvalues stored
	part_sum=0
	temp_eigenval=eigenval
	# Elements of I is 1 for all eigenvalues saved, 0 else
	I=np.zeros(eigenval.shape)
	#print(eigenval)
	while part_sum<total*percent:
		# Find the current maximum eigenvalue
		maxval=np.amax(temp_eigenval)
		# Set element equal 1 for the corresponding current max eigenvalue 
		I+=np.where(temp_eigenval==maxval,1,0)
		# Set current maximum eigenvalue to -inf
		temp_eigenval=np.where(temp_eigenval==maxval,-np.inf,temp_eigenval)
		part_sum+=maxval
	# Let ind be global indices of the corresponding eigenvalues used
	ind=np.where(I==1)[0]
	# Let V be the reduced matrix containing only the eigenvectors corresponding to the saved eigenvalues
	V_temp = w[:,ind]
	V=np.zeros((row,len(ind)))
	# 0 valued rows for elements belonging to boundary points
	V[J,:]=V_temp
	#Make reduced basis
	reduced_basis=(basis[:,_] * V).sum(0)
	return eigenval[ind],V, reduced_basis

## Representation using reduced basis functions

As a result of this we can represent our function $u$ as a linear combination of the $n$ reduced basis functions $u=\sum_{i=1}^{n}\tilde{N_i} \tilde{u}_i=\mathbf{\tilde{N}^\top} \mathbf{\tilde{u}}$,   $\mathbf{\tilde{N}}$ where $\mathbf{\tilde{u}}=[\tilde{u_1},\tilde{u_2}, \dots, \tilde{u_n}]^\top$ is the coefficient vector of $u$ in the reduced basis. If we want to represent $u$ as a linear combination of our $m$ original basis functions $\mathbf{N}\in\Omega$ like $u=\sum_{i=1}^m N_i u_i=\mathbf{N}^\top\mathbf{u}$ where $\mathbf{u}=[u_1,u_2, \dots, u_n]^\top$ is the coefficient vector of $u$ in our original basis, we can do this by choosing $\mathbf{u}=\mathbf{V}\mathbf{\tilde{u}}$ 
hence $\mathbf{N^\top}\mathbf{u}=\mathbf{N^\top}\mathbf{V}\mathbf{\tilde{u}}=(\mathbf{V^\top}\mathbf{N})^\top\mathbf{\tilde{u}}=\mathbf{\tilde{N}^\top} \mathbf{\tilde{u}}$


## Reduced Poisson
$reduced\_poisson()$ works in the same fashion as $poisson()$, but as our new reduced basis functions have global support we need to do the part with the lifting function ourselves. As described above we solve the system with $u=u_0+u_g$, where $u_0$ is 0 on the boundary and $u_g$ fullfills the boundary conditions. We construct $u_g$ in the same fashion as we did with $cons$ earlier, seting all elements belonging to the internal equal to 0. Note that $u_g$ is now represented using the original basis, but this doesn't matter as we are not going to invert this resulting matrix. We compute the $A$-matrix $a(\tilde{N_i},\tilde{N_j})$ using the reduced system, and the same for the integral including $f$, i.e. the integral $F(\tilde{N})$.Now comes the difference, insert $u_g=\sum_{j=1}^mN_ju_{gj}$ into your weak formulation, where $m$ is the number of the original basis functions. Then assemble the resulting matrix $A_{u_g}=\{a(\tilde{N_i},N_j)\}$ for $i=1,\dots,n$, $n$ the number of reduced basis functions and $j=1,\dots,m$. Then we compute

\begin{equation}
    \mathbf{u_0}=A^{-1}(\mathbf{F}-\mathbf{A_{u_g}}\mathbf{u_g})
\end{equation}
Now we map $u_0$ to the original basis as described above by puting $\mathbf{u_0}=\mathbf{V}\tilde{\mathbf{u_0}}$ and we can compute the control points of our approximate solution $\mathbf{u}=\mathbf{u_0}+\mathbf{u_g}$, and our solution is $u(\xi,\eta)=\mathbf{N}^\top\mathbf{u}$

In [61]:
def reduced_poisson(p,q,domain,geom,basis,old_basis,V,f,bc):
	# p,q polynomial degree of basis functions in \xi-, \eta-direction respectively
	# geom gives the geometry for which the problem is solved
	# f is the rhs function
	# bc gives the boundary conditions
	# basis is here the reduced basis and old_basis is the original basis
	# Returns sol,matrix where sol is the approximate solution of the reduced system (lin. comb of basisfunctions using the controlpoints)
	# matrix is the stiffness matrix of the reduced system
	x,y=geom
	### Assemble
	#Assemble stiffness matrix
	grad = basis.grad(geom)
	outer = function.outer(grad,grad)
	integrand = outer.sum(-1)
	degree=np.max([p,q])
	gauss='gauss'+str(int(2*(degree))) #Removing 2 also seems to solve the problem
	matrix = domain.integrate(integrand, geometry=geom,ischeme=gauss)
	# b.c.
	# Since basis functions now have global support we need a liftling function
	# Represent this as the boundary conditions projected onto the old basis
	ug=domain.boundary.project(bc(x,y),onto=old_basis,geometry=geom,ischeme=gauss)
	# Since ug know is a lin.comb. of old basis functions, our matrix for ug will be a bit different from the one for u0 as u0 is represented using 
	# the reduced basis functions
	grad_old=old_basis.grad(geom)
	outer_ug=function.outer(grad,grad_old)
	integrand_ug=outer_ug.sum(-1)
	matrix_ug=domain.integrate(integrand_ug,geometry=geom, ischeme=gauss)
	# Set all nan values of ug = 0
	ug=np.where(np.isnan(ug),0,ug)
	# setup the lifting function term a(v,ug)
	a_ug=matrix_ug.toarray().dot(ug)
	# Compute rhs 
	rhs = domain.integrate(f(x,y)*basis,geometry=geom,ischeme=gauss)
	rhs=rhs-a_ug
	# compute u0=A^-1*(rhs)
	u0 = matrix.solve(rhs, solver='cg')
	# Convert u0 to using the original basis(old basis) as described in my notes
	u0=V.dot(u0)
	sol=old_basis.dot(u0+ug)
	return sol,matrix

## Solution using reduced basis functions


In [63]:
percent=0.99
eigenvalues, V, reduced_basis= reduce_basis(K, percent,I, basis)
# Solve problem with reduced basis
reduced_sol,reduced_K=reduced_poisson(p,q,domain,geom,reduced_basis,basis,V,f1,bc1)
filename = "Reduced_"+name
error=error_analysis(reduced_sol, bc1,domain, geom, filename, 1)
print("The L^2-error squared is ", error)

integrate > elem 24/225 (10%)
integrate > elem 72/225 (31%)
integrate > elem 156/225 (69%)
project > solving system > solving system using sparse direct solver
project > constrained 72/361 dofs, error 6.02e-07/area
integrate > elem 72/225 (31%)
integrate > elem 216/225 (95%)
solving system > solving system using sparse direct solver
elem_eval > created ndarray(18225,2), ndarray(18225)
elem_eval > created ndarray(18225,2), ndarray(18225)
elem_eval > created ndarray(18225,2), ndarray(18225)
Reduced_poisson_FEM_u_approx000.png
Reduced_poisson_FEM_u_exact000.png
Reduced_poisson_FEM_pointwise_error000.png
The L^2-error squared is  3.81628007912e-06


![Reduced_u_approx](Reduced_poisson_FEM_u_approx000.png)
![Reduced_u_exact](Reduced_poisson_FEM_u_exact000.png)
![Reduced_pointwise_error](Reduced_poisson_FEM_pointwise_error000.png)

## Convergence
As an quick experiment I modified the convergence function and produced 3 different convergence plots.Firstly all eigenvalues to create my basis functions to verify that the method works. The plot is presented bellow, and as we see we get convergence. Then I took 90% of the eigenvalues, only choosing the largest to create my basis functions and plottet det convergence, the results are presented in the second figure. Lastly I did the same, but I took 90% of the eigenvalues only choosing the smallest ones, the results are presented in the last figure.

All eigenvalues used to create the basis functions
![Convergence test 100%](testing_convergence_100.png)
90% of all eigenvalues only choosing the largest where used to create the basis functions
![Convergence test 90% largest](testing_convergence_desc.png)
90% of all eigenvalues only choosing the smallest where used to create the basis functions
![Convergence test 90% smallest](testing_convergence_asc.png)

Code used for choosing eigenvalues and making convergence plots (not done!)

In [2]:
def reduce_basis_modified(K, percent,J, basis, order):
	###### Made as an experiment where we only pick a percentage of the eigenvalues starting from the first eigenvalue, where
	###### the eigenvalues in fact is sorted in an ascending order!!!
	# Select eigenvectors from K corresponding to x% of the total sum of the eigenvalues, choosing only the largest eigenvalues
	# and return the corresponding eigenvectors in a matrix with a eigenvector for each column
	# J gives the indicies in the stiffness matrix corresponding to internal points
	K=K.toarray()
	# save the size of the original stiffness matrix
	row,col=K.shape
	# Remove rows and columns belonging to boundary points
	K=K[np.ix_(J,J)]
	eigenval,w=LA.eigh(K)
	if order == 'asc':
		pass
	elif order == 'desc':
		eigenval=eigenval[::-1]
		w=w[:,::-1]
	else:
		print("Order must be 'asc' or 'desc'!")
	temp_eigenval=eigenval
	# Elements of I is 1 for all eigenvalues saved, 0 else
	I=np.zeros(eigenval.shape)
	#last eigenvalue to be included
	last_value=int(np.ceil(percent*len(eigenval)))
	V_temp=w[:,0:last_value]
	V=np.zeros((row,last_value))
	# 0 valued rows for elements belonging to boundary points
	V[J,:]=V_temp
	#Make reduced basis
	reduced_basis=(basis[:,_] * V).sum(0)
	return eigenval[0:last_value-1],V, reduced_basis


def reduced_convergence(filename,physicalspace,f,bc,pmin=1,pmax=4,nmin=0,nmax=6):
	# plot the convergence of the error for increasing number of elements, {2^nmin-2^nmax}, for different polynomial degree pmin-pmax,
	# saves the results in a npz file, can be visualized using plot_convergence()
	# increasing n, m with the same amount at the same time for each step, same for p and q
	# pmin,pmax is the min/max degree of the basis functions
	# n+1 gives number of grid points in one direction, hence nmin, nmax is the min/max number of gridpartitions
	# notice n is a power of two, nmin, nmax gives the min and max powers
	# Have assumed bc to be the exact solution over the whole domain for now, use this on boundary and also as exact solution
	order='desc'
	percent=0.90
	vec=np.arange(nmin,nmax+1)
	n=2**vec
	m=n
	p=np.arange(pmin,pmax+1)
	q=np.arange(pmin,pmax+1)
	error=np.zeros((len(p),len(n)))
	label=np.empty((len(p),),dtype=object)
	for i in range(0,len(p)):
		label[i]="p=q="+str(p[i])
		for j in range(0,len(n)):
			#Domain and basis setup
			domain, grid_geom,geom,basis=setup_domain(n[j],m[j],p[i],q[i],physicalspace)
			# Solve problem (assemble matrix, solve system)
			sol,K,I=poisson(p,q,domain,geom,basis,f,bc)
			eigenvalues, V, reduced_basis= reduce_basis_modified(K, percent,I, basis,order)
			reduced_sol,reduced_K=reduced_poisson(p,q,domain,geom,reduced_basis,basis,V,f1,bc1)
			# compute L2 error
			error[i,j]=error_analysis(reduced_sol, bc,domain, geom)
	# Writing results to file
	np.savez(filename,n=n,error=error,label=label)

## Further investigation
Picking 90% of the eigenvalues only choosing the largest eigenvalues gives us the following energy coverages, i.e. $\frac{\mbox{sum of chosen eigenvalues}}{\mbox{sum of all eigenvalues}}$


\begin{array}{|c|c|c|c|c|}
\hline p(q) & n(m) & \mbox{total amount of eigenvalues} & \mbox{chosen amount of eigenvalues (90%)} & \mbox{energy coverages}  \\\hline
    1 & 1 & 0 & 0 & nan \\
    1 & 2 & 1 & 1 & 1.0\\
    1 & 4 & 9 & 9 & 1.0\\
    1 & 8 & 49 & 45 & 0.984389196564\\
    1 & 16 & 225 & 203 & 0.982887668825\\
    2 & 1 & 1 & 1 & 1.0\\
    2 & 2 & 4 & 4 & 1.0\\
    2 & 4 & 16 & 15 & 0.979597768025\\
    2 & 8 & 64 & 58 & 0.968937444032\\
    2 & 16 & 256 & 231 & 0.971096944985\\
    3 & 1 & 4 & 4 & 1.0\\
    3 & 2 & 9 & 9 & 1.0\\
    3 & 4 & 25 & 23 & 0.991561098679\\
    3 & 8 & 81 & 73 & 0.986508266255\\
    3 & 16 & 289 & 261 & 0.985612039837\\
    4 & 1 & 9 & 9 & 1.0\\
    4 & 2 & 16 & 15 & 0.999514044876\\
    4 & 4 & 36 & 33 & 0.997680006674\\
    4 & 8 & 100 & 90 & 0.99528724427\\
    4 & 16 & 324 & 292 & 0.994428102509\\ \hline  
\end{array}



and the convergence plot $n=m=\{1,2,4,8,16\}$
![Convergence test 90% largest eigenvalues](90p_largest_eigenvalues.png)
Picking 90% of the eigenvalues only choosing the smallest eigenvalues gives us the following energy coverages, i.e. $\frac{\mbox{sum of chosen eigenvalues}}{\mbox{sum of all eigenvalues}}$

\begin{array}{|c|c|c|c|c|}
\hline p(q) & n(m) & \mbox{total amount of eigenvalues} & \mbox{chosen amount of eigenvalues (90%)} & \mbox{energy coverages}  \\\hline
    1 & 1 & 0 & 0 & nan\\
1 & 2 & 1 & 1 & 1.0\\
1 & 4 & 9 & 9 & 1.0\\
1 & 8 & 49 & 45 & 0.81289353866\\
1 & 16 & 225 & 203 & 0.773362756358\\
2 & 1 & 1 & 1 & 1.0\\
2 & 2 & 4 & 4 & 1.0\\
2 & 4 & 16 & 15 & 0.84586755905\\
2 & 8 & 64 & 58 & 0.780395747651\\
2 & 16 & 256 & 231 & 0.771859233057\\
3 & 1 & 4 & 4 & 1.0\\
3 & 2 & 9 & 9 & 1.0\\
3 & 4 & 25 & 23 & 0.735482233763\\
3 & 8 & 81 & 73 & 0.697451970743\\
3 & 16 & 289 & 261 & 0.716196745233\\
4 & 1 & 9 & 9 & 1.0\\
4 & 2 & 16 & 15 & 0.727224425564\\
4 & 4 & 36 & 33 & 0.646971056782\\
4 & 8 & 100 & 90 & 0.605851404649\\
4 & 16 & 324 & 292 & 0.630374887201\\ \hline  
\end{array}

and the convergence plot for $n=m=\{1,2,4,8,16\}$
![Convergence test 90% smallest eigenvalues](90p_smallest_eigenvalues.png)